# Step 1

**PRD Development**

Generate project manager stories and user personas for an AI-powered internal hiring platform.
Develop PRD from user stories for performance evaluation dashboard.
What internal hiring capabilities do project managers need?

In [14]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd()))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, render_plantuml_diagram

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")



2025-11-06 06:46:57,599 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


**Problem Statement**

In [15]:
problem_statement = """
Create an AI-powered performance evaluation dashboard that streamlines the internal talent assessment process by providing project managers with a centralized platform for managing employee profiles, analyzing performance data, and generating AI-powered discussions for internal opportunity evaluation.

The platform serves as an interactive dashboard where:

1. **Project managers** can add and manage project manager profiles, add and manage employee profiles through modal forms, browse and select employees from the dashboard, and view comprehensive employee analytics with detailed skills metrics and performance data for internal role assessment

2. **Employee profiles** are displayed with structured data including skills, experience, education, and quantified performance metrics (projects delivered, quality scores, velocity ratings) to support data-driven performance evaluation and internal opportunity matching

3. **Project managers** can engage with AI-generated conversation simulations that provide contextual discussions about specific employees for performance reviews and internal opportunity assessment, with capabilities to continue conversations, inject team lead comments, and collaborate on evaluation decisions

The solution provides dynamic profile management through modal-based data entry forms, AI-powered data generation for testing and demos, resizable dashboard layouts for personalized workflows, and structured conversation tools that simulate realistic performance evaluation discussions.

Key focus areas:
- Multi-column dashboard interface with project manager management, employee management, profile viewing, and conversation generation
- Comprehensive employee profiling with quantified performance metrics and detailed tooltips for evaluation insights
- AI-generated conversation simulations for performance evaluation and internal opportunity assessment
- Dynamic form-based data entry with AI-assisted profile generation for both managers and employees
- Interactive conversation tools with comment injection and discussion continuation capabilities
- Responsive design with customizable column layouts and modal-based workflows
"""

In [16]:
# TODO: Create a string variable named 'features_prompt'.
# This prompt should ask the LLM to brainstorm features based on the problem_statement.
features_prompt = f"""
Based on the following problem statement, brainstorm and list comprehensive features for an AI-powered internal hiring platform:

PROBLEM STATEMENT:
{problem_statement}

Please provide a detailed list of features organized by functional areas. Consider features for:
- User management and authentication (project managers only)
- Employee profile viewing and assessment (manager-controlled, employees cannot edit their own profiles)
- Project manager and hiring team workflow tools
- AI-powered internal talent matching and insights
- Collaboration and communication tools
- Analytics and reporting
- Data management and integration

IMPORTANT: This is a management dashboard where project managers evaluate employees for internal opportunities. Employees cannot edit their own information - all employee data is managed by project managers.

For each feature, provide:
1. Feature name
2. Brief description
3. Primary user beneficiary (Project Manager, Hiring Manager, HR Team) - NO employee self-service features
4. Priority level (High, Medium, Low)

Format your response as a structured list with clear categories.
"""

print("--- Brainstorming Features ---")
brainstormed_features = get_completion(features_prompt, client, model_name, api_provider)
print(brainstormed_features)

# TODO: Create a string variable named 'personas_prompt'.
# This prompt should ask the LLM to identify three user personas based on the problem_statement.
personas_prompt = f"""
Based on the following problem statement, identify and define three distinct user personas for an AI-powered internal hiring platform:

PROBLEM STATEMENT:
{problem_statement}

IMPORTANT: This is a management dashboard where project managers evaluate employees for internal opportunities. Employees cannot edit their own information and are not active users of the system - they are subjects of evaluation.

For each persona, provide:
1. Persona Name and Job Title
2. Demographics (age range, experience level, company size they typically work for)
3. Goals and Motivations (what they want to achieve using the platform)
4. Pain Points and Challenges (current problems they face in internal hiring/talent mobility)
5. Technology Comfort Level (how comfortable they are with AI and digital tools)
6. Key Use Cases (how they would primarily use the platform)
7. Success Metrics (how they would measure success when using the platform)
8. Required skills for project managers and acquired skills for employees

Focus on creating diverse personas that represent different aspects of the internal hiring management ecosystem, such as:
- A project manager who needs to identify talent for new projects
- A hiring manager or team lead who evaluates employees for internal opportunities
- An HR manager or talent mobility specialist who facilitates internal moves

Do NOT create employee personas as they are not active users of this management dashboard.

Make each persona realistic and detailed enough to guide product development decisions.
"""

print("\n--- Identifying User Personas ---")
user_personas = get_completion(personas_prompt, client, model_name, api_provider)
print(user_personas)

--- Brainstorming Features ---
Of course. Here is a comprehensive list of features for the AI-powered internal hiring platform, brainstormed and organized based on the provided problem statement.

### 1. User Management and Authentication
This area covers how project managers access and manage their own presence on the platform.

| Feature Name | Brief Description | Primary User Beneficiary | Priority |
| :--- | :--- | :--- | :--- |
| **Secure User Login** | Standard email/password authentication system for authorized managers. | Project Manager | High |
| **Single Sign-On (SSO) Integration** | Allows managers to log in using existing corporate credentials (e.g., Google, Microsoft, Okta) for seamless and secure access. | Project Manager, HR Team | High |
| **Role-Based Access Control (RBAC)** | Defines permissions for different user types (e.g., Project Manager sees their direct/indirect reports; Hiring Manager sees candidates for a role; HR Admin sees all). | HR Team, Project Manager 

**User Stories**

In [25]:
# TODO: Create a detailed prompt string named 'json_user_stories_prompt'.
# CREATE BOTH PROJECT MANAGERS AND EMPLOYEES USER STORIES?
json_user_stories_prompt = f"""
Act as a Senior Product Manager and generate user stories for an AI-powered internal hiring dashboard in STRICT JSON format.

CONTEXT:
PROBLEM STATEMENT:
{problem_statement}

USER PERSONAS (reference only):
{user_personas if 'user_personas' in locals() else 'No personas provided'}

BRAINSTORMED FEATURES (reference only):
{brainstormed_features if 'brainstormed_features' in locals() else 'No features provided'}

STRICT OUTPUT RULES:
1. Generate 12–14 user stories.
2. Allowed persona values (persona field only): "Project Manager", "Hiring Manager", "Talent Mobility Specialist", "System Administrator".
3. role field MUST exist and be ONLY: "Project Manager" (employees are NOT active users; use "Project Manager" for ALL stories).
4. Each story object MUST contain EXACTLY these fields IN THIS ORDER:
  "id" (integer, sequential starting at 1),
  "role" ("Project Manager"),
  "persona" (one of allowed personas),
  "user_story" ("As a [persona], I want [goal] so that [benefit]"),
  "acceptance_criteria" (array of 3–5 specific, testable strings),
  "Skills" (array of 6–10 employee skill domain categories ONLY)
5. Functional area coverage (each area >=1 story):
  - Project manager profile management & access control
  - Employee profile viewing & skills/performance assessment
  - AI-powered conversation simulations & continuation/injection
  - Modal-based data entry (add managers, add employees)
  - Dashboard layout customization (resizable/reorder columns, saved views)
  - Profile analytics (metrics, tooltips, comparison, filtering)
  - AI-assisted synthetic data generation (demo/prototype)
6. Acceptance criteria: Use explicit, testable conditions (Given/When/Then or clear functional checks). No vague wording.
7. Skills field formatting rules:
  - Provide ONLY employee skill / competency domains (e.g., Agile & Scrum Methodologies, Stakeholder Management, Roadmap Planning, Software Development Lifecycle (SDLC), Resource Allocation, Microservices Architecture, Cloud-Native Technologies, ROS 2, C++/Python, SLAM & Navigation, Machine Learning, Control Systems Design, Embedded System, SolidWork)
  - DO NOT include managerial capability labels (e.g., Decision Making, Bias Mitigation) and DO NOT duplicate concepts.
  - All items plain strings, no parentheses, no examples inline, no line breaks.
8. No extra keys, no explanations, no markdown fences, no trailing commas.
9. All strings must be plain (no line breaks inside array items).
10. Return a valid top-level JSON array.

SCHEMA EXAMPLE (FORMAT ONLY – DO NOT COPY CONTENT):
[
  {{
    "id": 1,
    "role": "Project Manager",
    "persona": "Project Manager",
    "user_story": "As a Project Manager, I want ... so that ...",
    "acceptance_criteria": [
      "Criterion 1",
      "Criterion 2",
      "Criterion 3"
    ],
    "Skills": [
      "React",
      "Python",
      "Recruiting"
    ]
  }}
]

Return ONLY the JSON array.
"""

print("--- Generating User Stories as JSON ---")
json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Let's try to parse the JSON to see if the LLM followed instructions
try:
    # The LLM might wrap the JSON in markdown fences (```json ... ```).
    # We'll clean that up before parsing.
    if '```' in json_output_str:
        json_output_str = json_output_str.split('```')[1].lstrip('json').strip()
    
    user_stories_json = json.loads(json_output_str)
    print("Successfully parsed LLM output as JSON.")
    
    if user_stories_json:
        print("\n--- Sample User Story ---")
        print(json.dumps(user_stories_json[0], indent=2))
    else:
        print("JSON array is empty.")

except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str)
    user_stories_json = []

--- Generating User Stories as JSON ---
Successfully parsed LLM output as JSON.

--- Sample User Story ---
{
  "id": 1,
  "role": "Project Manager",
  "persona": "System Administrator",
  "user_story": "As a System Administrator, I want to add new Project Manager profiles via a modal form so that I can securely provision access for new managers to the platform.",
  "acceptance_criteria": [
    "When I click the 'Add Manager' button, a modal form appears.",
    "The form must contain fields for Manager Name, Email, and Department.",
    "Upon successful submission, the new manager's profile appears in the manager list.",
    "An error message is displayed if the email format is invalid or already exists."
  ],
  "Skills": [
    "Agile & Scrum Methodologies",
    "Stakeholder Management",
    "Roadmap Planning",
    "Software Development Lifecycle (SDLC)",
    "Resource Allocation",
    "Budget Management"
  ]
}


**Validate Stories**

In [ ]:
def validate_and_save_stories(stories_data):
    """Validates the structure of the user stories data and saves it if valid."""
    if not isinstance(stories_data, list) or not stories_data:
        print("Validation Failed: Data is not a non-empty list.")
        return

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    all_stories_valid = True

    # TODO: Implement the validation logic inside this function.
    # 1. Loop through each story in the 'stories_data' list.
    # 2. For each story, check if it contains all the 'required_keys'.
    # 3. Also check if the 'acceptance_criteria' list is not empty.
    # 4. If a story is invalid, print an error message and set 'all_stories_valid' to False.
    #    (You can use 'continue' to skip to the next story).

    # Your validation code here
    for i, story in enumerate(stories_data, 1):
        # Check if story is a dictionary
        if not isinstance(story, dict):
            print(f"Error: Story {i} is not a dictionary.")
            all_stories_valid = False
            continue
            
        # Check if all required keys are present
        missing_keys = [key for key in required_keys if key not in story]
        if missing_keys:
            print(f"Error: Story {i} is missing required keys: {missing_keys}")
            all_stories_valid = False
            continue
            
        # Check if acceptance_criteria is not empty
        acceptance_criteria = story.get('acceptance_criteria')
        if not acceptance_criteria or (isinstance(acceptance_criteria, list) and len(acceptance_criteria) == 0):
            print(f"Error: Story {i} has empty acceptance_criteria.")
            all_stories_valid = False
            continue
            
        print(f"Story {i}: Valid")

    if all_stories_valid:
        print("\nAll user stories passed validation.")
        artifact_path = "artifacts/user_stories.json"
        
        # TODO: Call the save_artifact function from utils.py to save the data.
        # Remember to convert the Python list back to a JSON string using json.dumps().
        save_artifact(json.dumps(stories_data, indent=2), artifact_path, overwrite=True)
        print(f"User stories saved to {artifact_path}")
        
    else:
        print("\nValidation failed. Artifact not saved.")

# Run the validation on the JSON data from the previous step
if 'user_stories_json' in locals() and user_stories_json:
    validate_and_save_stories(user_stories_json)
else:
    print("Skipping validation as user_stories_json is empty or not defined.")

Story 1: Valid
Story 2: Valid
Story 3: Valid
Story 4: Valid
Story 5: Valid
Story 6: Valid
Story 7: Valid
Story 8: Valid
Story 9: Valid
Story 10: Valid
Story 11: Valid
Story 12: Valid
Story 13: Valid

All user stories passed validation.
User stories saved to artifacts/day1_user_stories.json


**Generate PRD**


In [ ]:
# Load the artifact from Lab 1
user_stories_str = load_artifact("user_stories.json")
if user_stories_str:
    user_stories_data = user_stories_str
else:
    user_stories_data = []


# TODO: Write a prompt to generate a simple PRD.
simple_prd_prompt = f"""
Generate a comprehensive Product Requirements Document (PRD) for an AI-powered internal hiring platform based on the following inputs:

PROBLEM STATEMENT:
{problem_statement}

USER STORIES DATA:
{json.dumps(user_stories_data, indent=2) if user_stories_data else 'No user stories available'}

BRAINSTORMED FEATURES (if available):
{brainstormed_features if 'brainstormed_features' in locals() else 'Generate features based on user stories'}

USER PERSONAS (if available):
{user_personas if 'user_personas' in locals() else 'Define personas based on user stories'}

Please create a well-structured PRD that includes the following sections:

1. **Product Overview**
   - Product name and brief description
   - Vision and mission statement
   - Target market and audience

2. **Problem Statement & Solution**
   - Clear problem definition
   - Proposed solution approach
   - Value proposition

3. **User Personas & Use Cases**
   - Primary user types and their characteristics
   - Key use cases and user journeys

4. **Functional Requirements**
   - Core features and capabilities organized by category
   - User stories mapped to functional requirements
   - Feature priorities (Must-have, Should-have, Could-have)

5. **Technical Requirements**
   - Platform and technology considerations
   - Performance requirements
   - Security and privacy requirements
   - Integration needs

6. **Success Metrics**
   - Key Performance Indicators (KPIs)
   - User engagement metrics
   - Business metrics

7. **Implementation Roadmap**
   - High-level development phases
   - MVP scope and timeline
   - Future enhancements

8. **Risks and Mitigation**
   - Technical risks
   - Business risks
   - Mitigation strategies

Make the PRD comprehensive but concise, focusing on the AI-powered internal hiring platform domain. Ensure all content aligns with the user stories and supports the overall product vision.

Format the output in clear markdown with proper headings and structure.
"""

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

--- Generating Simple PRD ---
Of course. Here is a comprehensive Product Requirements Document (PRD) for the AI-powered internal hiring platform, synthesized from the provided inputs.

---

## **Product Requirements Document: In-Sight AI Talent Platform**

**Version:** 1.0
**Date:** October 26, 2023
**Status:** Draft
**Author:** Product Management

---

### 1. Product Overview

#### 1.1 Product Name
In-Sight AI Talent Platform

#### 1.2 Brief Description
In-Sight is an AI-powered performance evaluation and internal mobility dashboard designed for project and hiring managers. It streamlines talent assessment by providing a centralized platform to manage employee profiles, analyze quantified performance data, and leverage AI-generated discussions to make objective, data-driven decisions about internal opportunities.

#### 1.3 Vision and Mission Statement
*   **Vision:** To unlock the full potential of every employee by creating a transparent, fair, and data-driven internal mobility ecosy

In [28]:
# Load the PRD template directly using the project root path
template_path = os.path.join(project_root, "templates", "prd_template.md")
try:
    with open(template_path, 'r', encoding='utf-8') as f:
        prd_template_content = f.read()
    print(f"Successfully loaded template from: {template_path}")
except FileNotFoundError:
    print(f"Template file not found at: {template_path}")
    prd_template_content = ""
except Exception as e:
    print(f"Error loading template: {e}")
    prd_template_content = ""

# TODO: Write a prompt to populate the PRD template.
template_prd_prompt = f"""
You are a Senior Product Manager at a technology company tasked with creating a comprehensive Product Requirements Document (PRD) for an AI-powered internal hiring platform. 

You have been provided with a formal PRD template that must be followed exactly. Your job is to populate every section of this template with detailed, professional content based on the user stories and project context provided.

CONTEXT PROVIDED:
USER STORIES DATA:
{json.dumps(user_stories_data, indent=2) if user_stories_data else 'No user stories available'}

PROBLEM STATEMENT:
{problem_statement}

SIMPLE PRD OUTPUT (reference material):
{simple_prd_output if 'simple_prd_output' in locals() and simple_prd_output else 'No simple PRD available'}

BRAINSTORMED FEATURES (if available):
{brainstormed_features if 'brainstormed_features' in locals() else 'Generate features based on user stories'}

USER PERSONAS (if available):
{user_personas if 'user_personas' in locals() else 'Generate personas based on user stories'}

PRD TEMPLATE TO POPULATE:
{prd_template_content}

INSTRUCTIONS:
1. **Follow the template structure exactly** - Do not add, remove, or reorder sections
2. **Fill out every section completely** - No section should be left blank or incomplete
3. **Use professional product management language** - Write as a Senior Product Manager would
4. **Base content on user stories** - Map user stories to functional requirements and features
5. **Infer logical content for template sections not covered in user stories**:
   - For "Success Metrics": Create measurable KPIs relevant to internal hiring platforms
   - For "Out of Scope": Define what's NOT included in V1.0 based on project complexity
   - For "Non-Functional Requirements": Infer performance, security, and scalability needs
   - For "Release Plan": Create realistic milestones based on feature complexity
   - For "Open Questions": Identify dependencies and decisions needed for an AI internal hiring platform

6. **Ensure content is specific to AI-powered internal hiring** - All examples, metrics, and features should be relevant to the internal talent mobility domain
7. **Replace ALL placeholder text** - Every [bracketed placeholder] and example text must be replaced with actual content
8. **Maintain table formatting** - Keep all tables properly formatted with actual data
9. **Be comprehensive but concise** - Provide enough detail for development teams while keeping sections focused

SPECIFIC REQUIREMENTS:
- Product name should reflect the AI-powered internal hiring focus
- User personas should be internal hiring-specific (Project Managers, Hiring Managers, HR Teams)
- Success metrics should include internal hiring KPIs like time-to-fill internal positions, employee mobility rates, user adoption rates
- Functional requirements should organize user stories into logical epics and features
- Technical requirements should address AI/ML infrastructure, real-time collaboration, and data security
- Out of scope should logically exclude advanced features not suitable for MVP

OUTPUT:
Return the complete PRD with every section of the template filled out. Maintain exact template formatting, structure, and table layouts while replacing all content with internal hiring platform-specific information.
"""

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

# Save the generated PRD to an artifact
if prd_from_template_output:
    save_artifact(prd_from_template_output, "project_prd.md", overwrite=True)

Successfully loaded template from: c:\aiswe\220372-AG-AISOFTDEV-Team-2-CodeVoyagers\templates\prd_template.md
--- Generating PRD from Template ---
# Product Requirements Document: In-Sight AI Talent Platform

| Status | **Draft** |
| :--- | :--- |
| **Author** | Product Management |
| **Version** | 1.0 |
| **Last Updated** | October 26, 2023 |

## 1. Executive Summary & Vision
In-Sight is an AI-powered performance evaluation and internal mobility dashboard designed for project managers, hiring managers, and talent specialists. It solves the critical business problem of inefficient and biased internal talent discovery by consolidating fragmented employee data into a single, interactive platform. By leveraging quantified performance metrics and AI-generated conversational insights, In-Sight empowers managers to make faster, fairer, and more data-driven decisions about internal opportunities.

Our vision is to unlock the full potential of every employee by creating a transparent, fair, an

## Step 2

**Generate design and architecture


In [29]:
# TODO: Write a prompt to generate a markdown ADR template.
adr_template_prompt = """
You are a Senior Software Architect tasked with creating a standardized template for Architectural Decision Records (ADRs) that will be used across software development projects.

Please generate a clean, professional markdown template for an Architectural Decision Record that includes the following sections:

1. **Title** - A concise, descriptive title for the architectural decision
2. **Status** - Current status of the decision (e.g., Proposed, Accepted, Deprecated, Superseded)
3. **Context** - Background information describing the problem, forces at play, and circumstances that led to this decision
4. **Decision** - The chosen solution, approach, or architectural choice being documented
5. **Consequences** - Both positive and negative outcomes/implications of this decision

REQUIREMENTS:
- Use proper markdown formatting with clear section headers
- Include placeholder text or brief descriptions for each section to guide users
- Make it professional and suitable for enterprise software development
- Ensure the template is reusable across different types of architectural decisions
- Include any metadata fields that would be helpful (date, decision makers, etc.)
- Keep the format clean and easy to follow

The template should be comprehensive enough to capture important architectural decisions while remaining simple enough for developers to use consistently.

OUTPUT: Return only the markdown template content without any additional explanation or wrapping.
"""

print("--- Generating ADR Template ---")
adr_template_content = get_completion(adr_template_prompt, client, model_name, api_provider)
print(adr_template_content)

# Save the artifact (overwrite if it already exists to avoid ArtifactError)
if adr_template_content:
    save_artifact(adr_template_content, "adr_template.md", overwrite=True)

--- Generating ADR Template ---
```markdown
# ADR-XXX: [Short descriptive title of the decision]

- **Status**: Proposed | Accepted | Deprecated | Superseded by [ADR-XXX](ADR-XXX.md)
- **Date**: YYYY-MM-DD
- **Decision Makers**: [List of names or roles]

---

## 1. Context

This section describes the problem, the context, and the forces at play that necessitated this decision. It should be a clear and concise summary of the issue being addressed.

- **Problem Statement**: What is the specific problem or challenge that needs a decision? What is the scope of this decision?
- **Driving Forces**: What are the key architectural drivers, constraints, requirements, or quality attributes influencing this decision? (e.g., performance, security, cost, maintainability, team skills, time-to-market).
- **Alternatives Considered**: Briefly list other options that were evaluated before arriving at this decision.

---

## 2. Decision

This section clearly and concisely describes our solution to the pr

In [30]:
from datetime import date

adr_template = load_artifact("artifacts/adr_template.md")

# TODO: Write a prompt to synthesize the final ADR.
today = date.today().isoformat()

synthesis_prompt = f"""
You are a Staff Engineer at a technology company responsible for making critical architectural decisions for an AI-powered internal hiring platform prototype project.

Your task is to create a formal Architectural Decision Record (ADR) documenting the decision to use SQLite as the database for the internal hiring platform prototype. You must use the provided ADR template and populate it with a well-reasoned justification for this choice.

PROJECT CONTEXT:
The AI-powered internal hiring platform is a one-day prototype that requires a database solution that can handle:
- Multi-column dashboard interfaces with real-time collaboration
- Employee profiles with skills metrics and conversation histories for internal opportunities
- Rapid development and deployment for proof-of-concept
- Simple setup with minimal operational overhead
- Local development and demo capabilities

DECISION MADE: SQLite has been chosen as the database technology for this prototype.

ADR TEMPLATE TO POPULATE:
{adr_template if adr_template else 'No template available - create a standard ADR structure'}

INSTRUCTIONS:
1. **Follow the ADR template structure exactly** - Use the provided template as your framework
2. **Act as a Staff Engineer** - Write with technical authority and clear reasoning
3. **Document the SQLite decision** - Justify why SQLite is the right choice for this prototype
4. **Focus on prototype requirements** - Emphasize rapid development, simplicity, and proof-of-concept needs
5. **Be comprehensive but concise** - Cover all template sections thoroughly
6. **Include today's date** - Use {today} as the decision date
7. **Address the internal hiring platform context** - Ensure the decision aligns with prototype requirements

SPECIFIC REQUIREMENTS:
- **Title**: Create a clear ADR title about choosing SQLite for the internal hiring platform prototype
- **Status**: Mark as "Accepted" since this is the formal decision
- **Context**: Explain the prototype requirements and why a simple database solution is needed
- **Decision**: State SQLite as the chosen approach with clear rationale focusing on:
  - Rapid prototype development (one-day timeline)
  - Zero-configuration setup and deployment
  - Local development and demo capabilities
  - Sufficient functionality for proof-of-concept
  - Easy migration path to production databases later
- **Consequences**: Detail both positive and negative implications, including:
  - Positive: Fast setup, no server management, portable, perfect for demos
  - Negative: Limited concurrent users, no built-in real-time features, eventual migration needed

The ADR should demonstrate pragmatic engineering judgment appropriate for a prototype project and provide clear guidance for the development team.

OUTPUT: Return the complete ADR document following the template structure, ready to be saved as the official architectural decision record.
"""

print("--- Synthesizing Final ADR ---")
if adr_template:
    final_adr = get_completion(synthesis_prompt, client, model_name, api_provider)
    print(final_adr)
    save_artifact(final_adr, "artifacts/adr_001_database_choice.md", overwrite=True)
else:
    print("Skipping ADR synthesis because template is missing.")

--- Synthesizing Final ADR ---
# ADR-001: Use SQLite for the Internal Hiring Platform Prototype Database

- **Status**: Accepted
- **Date**: 2025-11-06
- **Decision Makers**: Staff Engineer, Lead Engineer

---

## 1. Context

This section describes the problem, the context, and the forces at play that necessitated this decision. It should be a clear and concise summary of the issue being addressed.

- **Problem Statement**: The AI-powered internal hiring platform requires a database for its initial one-day prototype. The chosen solution must support rapid development, require zero configuration, and enable easy local demonstration without imposing any operational or infrastructure overhead. The scope of this decision is strictly limited to the proof-of-concept (PoC) phase.

- **Driving Forces**:
    - **Time-to-Market**: The primary constraint is the one-day development timeline. Any solution that requires significant setup, configuration, or a learning curve is unacceptable.
    - **D

# Generate Component Diagram

In [37]:
# TODO: Write a plantuml prompt for the component diagram
component_diagram_prompt = """
Create a simple, readable PlantUML component diagram for the performance evaluation dashboard that matches the actual implementation.

ACTUAL IMPLEMENTATION COMPONENTS (based on main.py FastAPI backend and frontend templates):

**WEB LAYER:**
- [Dashboard UI] - Main performance evaluation interface (index.html)
- [Modal Forms] - Add manager/employee forms with AI generation
- [Profile Display] - Employee profile viewer with analytics

**BACKEND LAYER (FastAPI - main.py):**
- [FastAPI App] - Web server with API endpoints (/api/employees, /api/project-managers, /api/conversation)
- [AI Generator] - LLM integration for generating realistic manager/employee data
- [Conversation Engine] - Manages AI-powered evaluation discussions between managers and employees

**DATA LAYER:**
- [SQLite Database] - Employee and project manager storage via utils.database
- [JSON Storage] - Conversation persistence in artifacts/conversations.json

**EXTERNAL:**
- [LLM Service] - Multiple providers (Google/OpenAI/Anthropic/HuggingFace) for AI content generation

REQUIREMENTS:
- Use PlantUML component syntax: [ComponentName]
- Show the actual FastAPI implementation with specific API endpoints
- Include the conversation engine as a separate component
- Show both SQLite and JSON storage layers
- Keep it readable but accurate to the real implementation
- Group components in logical packages
- If the LLM returns code fences, clean them.
- Ensure @startuml and @enduml are present (add if missing).

Return valid PlantUML code starting with @startuml and ending with @enduml.
"""

print("--- Generating Component Diagram ---")
component_puml_raw = get_completion(component_diagram_prompt, client, model_name, api_provider)
component_puml = clean_llm_output(component_puml_raw, language='plantuml')

def _ensure_uml_bounds(text: str) -> str:
    stripped = text.strip()
    if "@startuml" not in stripped:
        stripped = "@startuml\n" + stripped
    if "@enduml" not in stripped:
        stripped = stripped.rstrip() + "\n@enduml"
    return stripped

def safe_render_plantuml_diagram(puml_text: str, output_png: str):
    """Attempt to render PlantUML; on failure, save .puml source as artifact."""
    puml_text = _ensure_uml_bounds(puml_text)
    try:
        render_plantuml_diagram(puml_text, output_png)
        print(f"Rendered diagram to {output_png}")
    except TypeError as te:
        print(f"TypeError during PlantUML rendering (likely API signature mismatch): {te}")
        fallback_path = output_png.replace(".png", ".puml")
        save_artifact(puml_text, fallback_path, overwrite=True)
        print(f"Saved PlantUML source instead to {fallback_path}")
    except Exception as e:
        # Handle SSL or other runtime errors
        if "SSL" in str(e):
            print(f"SSL issue encountered while rendering diagram: {e}")
        else:
            print(f"Unexpected error rendering PlantUML: {e}")
        fallback_path = output_png.replace(".png", ".puml")
        save_artifact(puml_text, fallback_path, overwrite=True)
        print(f"Saved PlantUML source instead to {fallback_path}. You can render it manually (e.g., using the PlantUML jar).")

# Render the diagram safely
if component_puml:
    safe_render_plantUML_text = _ensure_uml_bounds(component_puml)
    # Removed unsupported 'overwrite' argument to match function signature
    safe_render_plantuml_diagram(safe_render_plantUML_text, "artifacts/component_diagram.png")
else:
    print("No PlantUML content generated.")

--- Generating Component Diagram ---
SSL issue encountered while rendering diagram: PlantUML rendering failed after attempts [outfile: PlantUML.processes() got an unexpected keyword argument 'outfile'; outfile positional: PlantUML.processes() takes 2 positional arguments but 3 were given]: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)
Saved PlantUML source instead to artifacts/component_diagram.puml. You can render it manually (e.g., using the PlantUML jar).


# Generate Use Case Diagram

In [40]:
# TODO: Write a plantuml prompt for the use case diagram  
use_case_diagram_prompt = """
Create a simple, readable PlantUML use case diagram for the performance evaluation dashboard.

ACTORS (with colors):
- Project Manager #Blue - Main user evaluating employees
- Employee #Green - People being evaluated
- AI System #Red - Provides AI assistance

CORE USE CASES (simplified for readability):
- Manage Dashboard
- Add Employee
- View Employee Profile
- Generate Discussion
- Customize Layout

REQUIREMENTS:
- Use PlantUML syntax: (Use Case Name) for use cases
- Color actors as specified above
- Put use cases inside "Performance Dashboard" system boundary
- Keep it very simple - only 5 main use cases
- Focus on readability over completeness
- Make the diagram clean and uncluttered

Return valid PlantUML code starting with @startuml and ending with @enduml.
"""

print("--- Generating Use Case Diagram ---")
use_case_puml_raw = get_completion(use_case_diagram_prompt, client, model_name, api_provider)
use_case_puml = clean_llm_output(use_case_puml_raw, language='plantuml')

# Robust rendering with fallback to saving .puml if PlantUML library / SSL issues occur
if use_case_puml:
    use_case_puml_text = use_case_puml.strip()
    if "@startuml" not in use_case_puml_text:
        use_case_puml_text = "@startuml\n" + use_case_puml_text
    if "@enduml" not in use_case_puml_text:
        use_case_puml_text = use_case_puml_text.rstrip() + "\n@enduml"
    try:
        # Attempt to render; if library signature mismatch or SSL failure occurs, handle gracefully
        render_plantuml_diagram(use_case_puml_text, "artifacts/use_case_diagram.png")
        print("Rendered use case diagram to artifacts/use_case_diagram.png")
    except TypeError as te:
        # Handle signature mismatch (e.g., unexpected kwargs or positional arg errors)
        print(f"TypeError during PlantUML rendering: {te}")
        fallback_path = "artifacts/use_case_diagram.puml"
        save_artifact(use_case_puml_text, fallback_path, overwrite=True)
        print(f"Saved PlantUML source instead to {fallback_path}")
    except Exception as e:
        # Handle SSL or other runtime errors
        print(f"Error rendering PlantUML use case diagram: {e}")
        fallback_path = "artifacts/use_case_diagram.puml"
        save_artifact(use_case_puml_text, fallback_path, overwrite=True)
        print(f"Saved PlantUML source instead to {fallback_path}. Render manually if needed.")
else:
    print("No PlantUML content generated.")

--- Generating Use Case Diagram ---
Error rendering PlantUML use case diagram: PlantUML rendering failed after attempts [outfile: PlantUML.processes() got an unexpected keyword argument 'outfile'; outfile positional: PlantUML.processes() takes 2 positional arguments but 3 were given]: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)
Saved PlantUML source instead to artifacts/use_case_diagram.puml. Render manually if needed.
